In [4]:
import pandas as pd
import numpy as np

In [5]:
# количество итераций обучения
NUM_OF_ITERATIONS = 50

# читаем размеченные данные из файла
beer_dataset = pd.read_excel('beer_data_set.xlsx')

# печатаем список колонок таблички чтобы удобнее было копировать
print('\n'.join(beer_dataset.columns))

# выводим несколько первых строк таблицы
beer_dataset.head()

barcode
SKU_NAME
Наименование
Алкоголь
Объем
Производитель
Бренд
Саб-бренд
Тип упаковки
Мультипак
тип


,barcode,SKU_NAME,Наименование,Алкоголь,Объем,Производитель,Бренд,Саб-бренд,Тип упаковки,Мультипак,тип
0,4600721002206,Пиво БагБир 0.5л ст/бут,BAGBIER - светлое - 4.2% - 0.5л стекло,0.042,0.5,AB InBev,BAGBIER,NaN,Стекло,NaN,Светлое
1,4600721003197,Пиво БАГ-БИР св.ст/б 0.5л,BAGBIER - светлое - 4.2% - 0.5л стекло,0.042,0.5,AB InBev,BAGBIER,NaN,Стекло,NaN,Светлое
2,4600721003203,"Пиво BAGBIER светлое 4,9% 1.5л",BAGBIER - светлое - 4.9% - 1.5л пэт,0.049,1.5,AB InBev,BAGBIER,NaN,ПЭТ,NaN,Светлое
3,4600721005191,Пиво БАГ-БИР св.ПЭТ 2.5л,BAGBIER - светлое - 4.2% - 2.5л пэт,0.042,2.5,AB InBev,BAGBIER,NaN,ПЭТ,NaN,Светлое
4,4600721009366,"Пиво БАГ БИР ГОЛЬДЕН светлое ПЭТ 4% 1,5л",BAGBIER GOLDEN - светлое - 4.7% - 1.5л пэт,0.047,1.5,AB InBev,BAGBIER,Golden,ПЭТ,NaN,Светлое


Импортируем преобразователь текста в вектор.

[Примерное объяснение как работает](http://zabaykin.ru/?p=463)

[Документация](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

Импортируем преобразователь меток в числа. 
Он присваивает каждому новой встреченной метке уникальный номер.

Допустим у нас есть такой набор меток:
```python
labels = ['заяц', 'волк', 'утка', 'белка', 'заяц', 'утка']
```

LabelEncoder строит примерно такое соответсвие:

| Метка | Номер |
|:-----:|:-----:|
|  волк |   0   |
| белка |   1   |
| заяц  |   2   |
| утка  |   3   |

Так что если к нам поступает такой набор меток,
```python
['утка', 'утка', 'заяц', 'утка', 'белка', 'волк', 'заяц', 'волк', 'волк', 'белка']
```
то мы можем преобразовать их в список чисел
```python
[3, 3, 2, 3, 1, 0, 2, 0, 0, 1]
```

Это нужно потому что многие алгоритмы не умеют работать со строками, к тому же числа занимаю меньше места

[Документация](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

In [7]:
from sklearn.preprocessing import LabelEncoder

Импортируем функцию для разбиения датасета на обучающий и тестирующий набор данных случайным образом. 

Зачем это нужно: когда модель обучается, то она может начать запоминать сочетания "вопрос"-"ответ". Вместо того, чтобы пытаться разобраться в во входных данных. Поэтому модель нужно проверять на данных, которых она до этого не видела. 

[Немного подробнее](http://robotosha.ru/algorithm/training-set-and-test-data.html)

[Документация](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [8]:
from sklearn.model_selection import train_test_split

Импортируем классификатор данных из библиотеки CatBoost от Яндекса

In [9]:
from catboost import CatBoostClassifier

Выдираем сырые строки из колонки **SKU_NAME**. В таблице строки хранятся в виде списков байт, метод `.flatten()` преобразует их в простые строки

In [10]:
data = [item for item in beer_dataset[['SKU_NAME']].values.flatten()]


Выдираем метки из колонки **Объём**. В начальном наборе слишком много меток, поэтому делаем кластер меток, в которые входит 95% самых популярных

In [95]:
from sklearn.cluster import KMeans
from collections import Counter 
from itertools import accumulate

raw_labels =  beer_dataset['Объем'].values.flatten()
p = list(accumulate(count/len(raw_labels) for _, count in Counter(raw_labels).items()))
top_labels = np.sort([item for i, item in enumerate(Counter(raw_labels))
                if p[i] <= 0.95])         

cluster_centers = top_labels.reshape(-1, 1)
kmeans = KMeans(n_clusters=top_labels.shape[0], n_init=1, init=cluster_centers)
kmeans.fit(raw_labels.reshape(-1, 1))
labels = [kmeans.cluster_centers_[c][0] for c in kmeans.predict(raw_labels.reshape(-1, 1))]
pd.DataFrame({'volume':raw_labels, 'class':labels}, columns=['volume', 'class']).head(10)

,volume,class
0,0.50,0.500000
1,0.50,0.500000
2,1.50,1.500153
3,2.50,2.569767
4,1.50,1.500153
5,2.50,2.569767
6,1.50,1.500153
7,0.50,0.500000
8,0.50,0.500000
9,0.33,0.329786


Разбиваем датасет случайным образом на обучающие и тестовые данные с соотношением 2 к 1. `random_state` – это начальное состояние генератора случайных чисел для разбиения, ставим число 42 чтобы каждый раз разбиение было одним и тем же.

In [40]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.33, random_state=42)

Печатаем статистику полученных датасетов: суммарный размер (Total), размер обучающей выборки (train) и тестовой (test)

In [41]:
print(f"Total: {len(data)} samples\n"
     f"\ttrain: {len(train_data)} data, {len(train_labels)} labels\n"
     f"\ttest: {len(test_data)} data, {len(test_labels)} labels")

Total: 5824 samples
	train: 3902 data, 3902 labels
	test: 1922 data, 1922 labels


Воспомогательная функция. CountVectorizer возвращает сжатые вектора, а нам нужны обычные. Эта функция берёт список сжатых веткоров и преобразует их в массивы чисел.

In [42]:
def dense_vectors(vectors):
    return [np.asarray(item.todense())[0] for item in vectors]

Функция обучения модели. Наша модель состоит из трёх частей:
    - `CountVectorizer` для преобразования входных данных в векторное представление
    - `LabelEncoder` для преобразования меток в числа
    - `CatBoostClassifier` – собственно классификатор данных

По-хорошему надо сохранять все три части в один или несколько файлов, пока для тестирования сохраняем только последнее. 

Входные данные в виде списка строк переводятся в нижний регистр, разбиваются на токены с помощью регулярного выражения `(?u)\b\w\w+\b|[0-9\.,]+[\%\w]|\w+`. Посмотреть как работает это выражение можно посмотреть [здесь](https://regex101.com/r/Puyk9J/1). Оно разбивает строки на список подстрок примерно так


|Строка| Токены |
|:------------------------------|:-----------------------------------:|
| Пиво БагБир 0.5л  ст/бут | ['Пиво', 'БагБир', '0.5л', 'ст', 'бут'] |
| Пиво БАГ-БИР св.ст/б 0.5л | ['Пиво', 'БАГ', 'БИР', 'св', '.с', 'т', 'б', '0.5л'] |
| Пиво BAGBIER светлое 4,9%   1.5л | ['Пиво', 'BAGBIER', 'светлое', '4,9%', '1.5л'] |
| Пиво БАГ-БИР св.ПЭТ 2.5л | ['Пиво', 'БАГ', 'БИР', 'св', '.П', 'ЭТ', '2.5л'] |
| Пиво БАГ БИР ГОЛЬДЕН светлое ПЭТ 4% 1,5л | ['Пиво', 'БАГ', 'БИР', 'ГОЛЬДЕН', 'светлое', 'ПЭТ', '4%', '1,5л'] |
| Пиво БАГ-БИР ГОЛЬДЕН св.4% ПЭТ 2.5л | ['Пиво', 'БАГ', 'БИР', 'ГОЛЬДЕН', 'св', '.4%', 'ПЭТ', '2.5л'] |
| Пиво БАГ БИР БОК темное 4% 1.5л | ['Пиво', 'БАГ', 'БИР', 'БОК', 'темное', '4%', '1.5л'] |
| Нап.пивн.BAGBIER 4,6% ст/б 0.5л | ['Нап', '.п', 'ивн', '.B', 'AGBIER', '4,6%', 'ст', 'б', '0.5л'] |
| Нап.пивн.БАГ БИР 4,2% ст/б 0.5л | ['Нап', '.п', 'ивн', '.Б', 'АГ', 'БИР', '4,2%', 'ст', 'б', '0.5л'] |
| Пиво BRAHMA 4.6% св.ст/б  0.33л | ['Пиво', 'BRAHMA', '4.6%', 'св', '.с', 'т', 'б', '0.33л'] |

Потом мы переводим данные в сжатые вектора, а из них получаем простые вектора с помощью фунции `dense_vectors`. Затем создаём LabelEncoder() и заставляем его запомнить наши метки.

Теперь можно заняться самым главным: обучить классификатор. 

Мы создаём CatBoostClassifier с настраиваемым параметром iterations (число итераций) и обучаем его через вызов метода `fit` ([документация](https://catboost.ai/docs/concepts/python-reference_catboostclassifier_fit.html)). Этот метод принимает обучающие данные в виде списка векторов (`vectorized_data`) и список закодированных меток.

После того, как обучились, возвращаем кортеж вида `(CountVectorizer, LabelEncoder, CatBoostClassifier)`

In [89]:
import re 
import random
import time

random.seed(time.time())
analyzer = 'word'
token_pattern = r"(?u)\b\w\w+\b|[0-9\.,]+[\%\w]|\w+"

def tokenize(items, token_pattern):
    pat = re.compile(token_pattern)
    tokens = []
    for item in items:
        tokens.append([match.group() for match in re.finditer(pat, item)])
    data = zip(items, tokens)
    return pd.DataFrame(data, columns=['Строка', 'Токены'])

tokenize(random.sample(data, 10), r"(?m)[a-zA-Zа-яА-Я]+")


,Строка,Токены
0,"Пиво БОЧКАРЕВ св.4,7% ст/б 0.5л","[Пиво, БОЧКАРЕВ, св, ст, б, л]"
1,"Пиво ДОН НЕФИЛЬТР.св.4,6% ст/б 0.47л","[Пиво, ДОН, НЕФИЛЬТР, св, ст, б, л]"
2,Пиво BELHAV.СКОТТИШ СТАУТ тем.7% 0.5л,"[Пиво, BELHAV, СКОТТИШ, СТАУТ, тем, л]"
3,KROMBACHER HELL 1L NF L F,"[KROMBACHER, HELL, L, NF, L, F]"
4,Пиво БЕЛЫЙ МЕДВЕДЬ св.ж/б 0.5л,"[Пиво, БЕЛЫЙ, МЕДВЕДЬ, св, ж, б, л]"
5,ZOLOTOY CHEKH 0.5L J BO L X,"[ZOLOTOY, CHEKH, L, J, BO, L, X]"
6,"Пиво РУССКОЕ ПИВО клас.св.паст.4,0% 1.0л","[Пиво, РУССКОЕ, ПИВО, клас, св, паст, л]"
7,KRUSOVICE - светлое 4.2% - 0.5л ж.б.,"[KRUSOVICE, светлое, л, ж, б]"
8,ZHIGULYOVSK.(TOMSKOE PIVO) FIRMENNOE 0.5L J BO...,"[ZHIGULYOVSK, TOMSKOE, PIVO, FIRMENNOE, L, J, ..."
9,Пиво ВЕЛКОПОПОВИЦКИЙ КОЗЕЛ св.кега 4% 1л,"[Пиво, ВЕЛКОПОПОВИЦКИЙ, КОЗЕЛ, св, кега, л]"


In [83]:

def build_model(data, labels, iterations=200):
        vectorizer = CountVectorizer(lowercase=True)
        
        compressed_data = vectorizer.fit_transform(data)
        vectorized_data = dense_vectors(compressed_data)
        
        le = LabelEncoder()
        encoded_labels = le.fit_transform(labels)
            
        classifier = CatBoostClassifier(iterations=iterations, task_type = "GPU")
        classifier.fit(vectorized_data, encoded_labels, silent=False)
        
        return (vectorizer, le, classifier)


Обучаем нашу модель

In [84]:
model = build_model(train_data, train_labels, iterations=NUM_OF_ITERATIONS)

0:	learn: -5.9044050	total: 844ms	remaining: 41.4s
1:	learn: -5.7587951	total: 1.8s	remaining: 43.3s
2:	learn: -5.6491558	total: 2.68s	remaining: 42s
3:	learn: -5.5603306	total: 3.55s	remaining: 40.9s
4:	learn: -5.4850888	total: 4.4s	remaining: 39.6s
5:	learn: -5.4207348	total: 5.24s	remaining: 38.4s
6:	learn: -5.3619938	total: 6.1s	remaining: 37.5s
7:	learn: -5.3102355	total: 6.96s	remaining: 36.6s
8:	learn: -5.2630427	total: 7.8s	remaining: 35.6s
9:	learn: -5.2185368	total: 8.64s	remaining: 34.6s
10:	learn: -5.1758879	total: 9.49s	remaining: 33.6s
11:	learn: -5.1347391	total: 10.3s	remaining: 32.7s
12:	learn: -5.0962162	total: 11.2s	remaining: 31.8s
13:	learn: -5.0641718	total: 12.1s	remaining: 31.1s
14:	learn: -5.0316920	total: 12.9s	remaining: 30.1s
15:	learn: -5.0002873	total: 13.8s	remaining: 29.2s
16:	learn: -4.9741068	total: 14.6s	remaining: 28.4s
17:	learn: -4.9451884	total: 15.4s	remaining: 27.5s
18:	learn: -4.9158104	total: 16.3s	remaining: 26.6s
19:	learn: -4.8894863	total:

Генератор отчёта по нашей модели. Скармливаем ей модель, тестовые данные и метки.

Метод `.predict` ([документация](https://catboost.ai/docs/concepts/python-reference_catboostclassifier_predict.html)) классификатора `CountVectorizer` принимает список векторов (входные данные) и возвращает для них самые вероятные ответы в виде чисел float, которые нужно преобразовать к целым числам.

Метод `.predict_proba` ([документация](https://catboost.ai/docs/concepts/python-reference_catboostclassifier_predict_proba.html)) классификатора `CountVectorizer` принимает список векторов (входные данные) и возвращает для них вероятности полученных выше ответов. Полученные вероятности умножем на 100 и преобразумем в строки вида `95%, 80%, 91%`. 

После этого берём полученные из метода `.predict` ответы и раскодируем их с помощью метода `.inverse_transform` кодировщика `LabelEncoder`, который преобразует список чисел в список строк.

Из полученных выше ответов, входных данных и правильных ответов из тестового датасета делаем табличку (`table`) и преобразуем её в удобный для нас вид `pandas.DataFrame`.

In [78]:
def validate_model(model, valid_data, valid_labels):
    vectorizer, le, classifier = model
    columns = ["Запись", "Предсказание", "Уверенность" ,"Правильный результат"]

    compressed_data = vectorizer.transform(valid_data)
    vectorized_data = dense_vectors(compressed_data)
    prediction = classifier.predict(vectorized_data).flatten().astype('int64')
    
    proba = np.rint(100*classifier.predict_proba(vectorized_data).max(axis=1))
    proba_column = (f"{int(item)}%" for item in proba)
    
    results = le.inverse_transform(prediction).flatten()
    table = zip(valid_data, results, proba_column, valid_labels)
    return pd.DataFrame(table, columns=columns)

Запускаем валидацию нашей модели и выводим сравнительную табличку результатов

In [86]:
validation = validate_model(model, test_data, test_labels)
validation.head(20)

,Запись,Предсказание,Уверенность,Правильный результат
0,"Пиво СТЕПАН РАЗИН ПЕТР.св.4,7% ПЭТ 1.4л",жигулевское,3%,степан разин
1,ZLATY BAZANT - светлое 4.1% - 0.5л ж.б.,жигулевское,3%,zlaty bazant
2,"Пиво MILLER MIDNIGHT тем.4,8% ст/б 0.5л",жигулевское,3%,miller
3,КЛИНСКОЕ - светлое 4.5% - 0.5л стекло,клинское,53%,клинское
4,"ПивоANDECH.ВАЙСБ.ХЕФЕТР.св.5,5%ст/б 0.5л",жигулевское,3%,andechser vollbier hell
5,ЖИГУЛЕВСКОЕ - светлое 4.2% - 1.5л пэт,жигулевское,56%,жигулевское
6,"Пиво ЯРПИВО КРЕПК.св. 7,2% 0.5л",ярпиво,32%,ярпиво
7,БАЛТИКА - светлое 5.4% - 0.5л стекло,балтика,71%,балтика
8,SIBIRSKAYA KORONA(AB INBEV) KLASSICHESKOE 0.5L...,жигулевское,3%,сибирская корона
9,OKHOTA(HEINEKEN BEL) KREPKOE EKSPORTNOE 1.4L P...,жигулевское,3%,охота


Хорошо бы посчитать процент правильных ответов. Фильтруем все строки, в которых значение в столбце **Предсказание** совпадает со значением в столбце **Правильный результат** и считаем их количество.

Делим число правильных ответов на размер тестового набора данных и выводим.

In [87]:
valid = len(validation[validation['Предсказание'] == validation['Правильный результат']])
total = len(validation)
print(f"Valid: {valid} from {total} ({100*valid/total}%)")

validation[validation['Предсказание'] != validation['Правильный результат']].head(20)

Valid: 563 from 1922 (29.292403746097815%)


,Запись,Предсказание,Уверенность,Правильный результат
0,"Пиво СТЕПАН РАЗИН ПЕТР.св.4,7% ПЭТ 1.4л",жигулевское,3%,степан разин
1,ZLATY BAZANT - светлое 4.1% - 0.5л ж.б.,жигулевское,3%,zlaty bazant
2,"Пиво MILLER MIDNIGHT тем.4,8% ст/б 0.5л",жигулевское,3%,miller
4,"ПивоANDECH.ВАЙСБ.ХЕФЕТР.св.5,5%ст/б 0.5л",жигулевское,3%,andechser vollbier hell
8,SIBIRSKAYA KORONA(AB INBEV) KLASSICHESKOE 0.5L...,жигулевское,3%,сибирская корона
9,OKHOTA(HEINEKEN BEL) KREPKOE EKSPORTNOE 1.4L P...,жигулевское,3%,охота
10,CLAUSTHALER - светлое 0% - 0.33л стекло,жигулевское,3%,clausthaler
14,"Нап.С&Р.ГАР.ХАРД БРУС.осн.пив.4,6% 0.44л",жигулевское,3%,seth & riley`s garage
15,CORONA EXTRA 6X0.355L J BO L F,жигулевское,3%,corona extra
16,"Пиво КЕР САРИ ТЕМНОЕ 4,3% ж/б 0.5л",жигулевское,3%,кер сари


Не забываем сохранить модель в файл!

In [81]:
vectorizer, le, classifier = model

model_name = f"beer_volume_catboost_{NUM_OF_ITERATIONS}"

classifier.save_model(f"{model_name}.cbm")


In [ ]:
from joblib import dump, load

dump(le, f"{model_name}_le.job")
dump(vectorizer, f"{model_name}_vect.job")